In [1]:
%load_ext autoreload
%autoreload 2
import torch
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.visualization.singleview import make_singleview_prediction_plots, filter_predictions
from cosypose.visualization.singleview import filter_predictions
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Setting OMP and MKL num threads to 1.
pybullet build time: Sep  9 2021 15:08:49


Loading BokehJS ...

In [2]:
result_id = 'tless-siso-n_views=1--4312481950'
ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
pred_key = 'pix2pose_detections/coarse/iteration=1'

results = LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar'
scene_ds = make_scene_dataset(ds_name)
results = torch.load(results)['predictions']

0:00:00.441811 - Building index and loading annotations...


[Memory]0.7s, 0.0min    : Loading build_index...


In [3]:
results[pred_key].infos.loc[:, ['scene_id', 'view_id']].groupby('scene_id').first()
results[pred_key].infos

,scene_id,view_id,score,label,det_id,batch_im_id,group_id
0,3,353,1.000000,obj_000005,5214,0,1087
1,3,353,0.922440,obj_000012,5215,0,1087
2,3,353,0.874073,obj_000008,5216,0,1087
3,3,353,0.497611,obj_000011,5217,0,1087
4,3,353,0.395497,obj_000018,5218,0,1087
...,...,...,...,...,...,...,...
50018,20,228,0.679904,obj_000003,49006,0,9845
50019,6,359,1.000000,obj_000012,11780,0,2732
50020,6,359,0.754588,obj_000006,11781,0,2732
50021,6,359,0.532693,obj_000011,11782,0,2732


In [5]:
scene_id, view_id = 18, 503

# Replace this here, you can use the dataframe above to get examples of scene/view ids.
# scene_id, view_id = 48, 733

this_preds = filter_predictions(results[pred_key], scene_id, view_id)
renderer = BulletSceneRenderer(urdf_ds_name)
figures = make_singleview_prediction_plots(scene_ds, renderer, this_preds)
renderer.disconnect()
# print(this_preds)

show(figures['input_im'])
show(figures['pred_overlay'])

Loaded EGL 1.5 after reload.
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=GeForce RTX 2060/PCIe/SSE2
GL_VERSION=4.6.0 NVIDIA 460.91.03
GL_SHADING_LANGUAGE_VERSION=4.60 NVIDIA
Version = 4.6.0 NVIDIA 460.91.03
Vendor = NVIDIA Corporation
Renderer = GeForce RTX 2060/PCIe/SSE2


EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)


ven = NVIDIA Corporation
Destroy EGL OpenGL window.
